In [35]:
import pandas as pd
pd.set_option('display.max_rows', 5000)
import numpy as np
import glob
import os

In [2]:
fixpath = 'D:/honda data/percarpercitydata/'
folders = [ name for name in os.listdir('D:/honda data/percarpercitydata/') if os.path.isdir(os.path.join('D:/honda data/percarpercitydata/', name)) ]

# paths = glob.glob('D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/')
folders

['Ahmedabad - 2016 Oct',
 'Amritsar - 2016 Jan',
 'Bangalore - 2016 Oct',
 'Calcutta - 2016 Jun',
 'Chennai - 2016 Oct',
 'Delhi (New Data) - 2016 Dec',
 'Delhi - 2015 Nov',
 'Jalandhar - 2016 Feb',
 'Kanpur - 2016 Apr',
 'Lucknow - 2016 May',
 'Ludhiana - 2016 Mar',
 'Mumbai - 2015 Dec',
 'Nagpur - 2016 Jan',
 'Patna - 2016 Jul',
 'Pune - 2015 Dec']

In [120]:
k = {}
for city in folders:
    cars = [ name for name in os.listdir(fixpath + city) if os.path.isdir(os.path.join(fixpath + city, name)) ]
    for y in cars:
        paths = glob.glob(fixpath + city + "/" + y + '/*.csv')
        maxSize = 0
        for ind, file in enumerate(paths):
            prev = os.stat(file).st_size
            if maxSize < prev:
                maxSize = prev
                finalInd = ind
        name = city.split(" ")
        k[name[0] + "_"+ y] = {}
        k[name[0] + "_"+ y]['path'] = paths[finalInd]

In [121]:
k

{'Ahmedabad_GJ01RF4950_Amaze': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ01RF4950_Amaze\\A20160704153807a.csv'},
 'Ahmedabad_GJ09BD3161_Mobilio': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ09BD3161_Mobilio\\A20160624120858a.csv'},
 'Ahmedabad_GJ20N0977_City': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ20N0977_City\\Z20160616162057a.csv'},
 'Amritsar_Amritsar-PB02CE6090_Amaze csv': {'path': 'D:/honda data/percarpercitydata/Amritsar - 2016 Jan/Amritsar-PB02CE6090_Amaze csv\\A20151223064036a.csv'},
 'Amritsar_Amritsar-PB02CH9400_City csv': {'path': 'D:/honda data/percarpercitydata/Amritsar - 2016 Jan/Amritsar-PB02CH9400_City csv\\A20160111143757a.csv'},
 'Amritsar_Amritsar-PB02T0015_Jazz csv': {'path': 'D:/honda data/percarpercitydata/Amritsar - 2016 Jan/Amritsar-PB02T0015_Jazz csv\\A20160112202400a.csv'},
 'Bangalore_KA03MU3992_Amaze': {'path': 'D:/honda data/percarpercitydata/Bangalore - 2016 Oct/KA03MU3992_Amaze\\Z20160

In [122]:
for keys, item in k.items():
    data = pd.read_csv(item['path'], skiprows = [0], header = [0])
    start = 0
    end = 0
    df = pd.DataFrame()
    for ind in range(data.shape[0]):
        if data['NE'].iloc[ind] > 0:
            start = ind
            break
        else:
            start = None
    for ind in range(data.shape[0] -1,0, -1):
        if data['NE'].iloc[ind] > 0:
            end = ind
            break
        else:
            end = None
    if start:
        data = data.drop(index = range(start))
    if end:
        data = data.drop(index = range(end, data.shape[0]))
    data = data.reset_index(drop = True)
    speed = []
    time=[]
    rpm = []
    for ind, x in enumerate(data['Smp.time']):
        if x % 1000 == 0 :
            speed.append(data['VSP'].iloc[ind])
            rpm.append(data['NE'].iloc[ind])
            time.append(x)
        else:
            continue
    df['speed'] = speed
    df['time'] = time
    df['rpm'] = rpm
    df = df.dropna()
    df['speed'] = df['speed']
    df['time'] = df['time']/1000
    speed_diff = df['speed'] - df['speed'].shift(1)
    time_diff = df['time'] - df['time'].shift(1)
    acc = (speed_diff/time_diff) * 0.0283                          # converting km/h-s to g
    ha = 0
    hb = 0
    idle_time = 0
    idle = 0
    s = 0
    for x in acc:
        if x > 0.4:
            ha = ha+1
        elif x < -0.6:
            hb = hb+1
    k[keys]['ha'] = ha
    k[keys]['hb'] = hb
    val = True
    for x in range(len(df['speed'])-1):
        if df['speed'][x] == 0 and df['rpm'][x] > 650:
            idle_time = idle_time+1
#             if val:
#             count = count+1
                
    k[keys]['total_idle_time'] = idle_time/60
#     k[keys]['idling_times'] = idle
    for x in df['speed']:
        if x>90:
            s = s+1
    k[keys]['overspeed'] = s/60
    
    k[keys]['fuel_used'] = (data['ENG_FUEL_USED'].iloc[-1] - data['ENG_FUEL_USED'].iloc[0])/1000
    k[keys]['Total_distance'] = data['METER_ODO_DATA'].iloc[-1] - data['METER_ODO_DATA'].iloc[0]
    k[keys]['Total_time'] = data['Smp.time'].iloc[-1]/(1000*60)
    k[keys]['Mileage'] = k[keys]['Total_distance']/k[keys]['Total_time']
    if k[keys]['Total_time'] > 360:
        k[keys]['fatigue_driving'] = 1
    else:
        k[keys]['fatigue_driving'] = 0
    
        
    

In [123]:
k

{'Ahmedabad_GJ01RF4950_Amaze': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ01RF4950_Amaze\\A20160704153807a.csv',
  'ha': 0,
  'hb': 0,
  'total_idle_time': 30.466666666666665,
  'overspeed': 0.0,
  'fuel_used': 1.4312,
  'Total_distance': 22.0,
  'Total_time': 82.655,
  'Mileage': 0.2661665960921904,
  'fatigue_driving': 0},
 'Ahmedabad_GJ09BD3161_Mobilio': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ09BD3161_Mobilio\\A20160624120858a.csv',
  'ha': 0,
  'hb': 0,
  'total_idle_time': 49.45,
  'overspeed': 0.0,
  'fuel_used': 1.3252000000000002,
  'Total_distance': 22.0,
  'Total_time': 91.965,
  'Mileage': 0.239221442940249,
  'fatigue_driving': 0},
 'Ahmedabad_GJ20N0977_City': {'path': 'D:/honda data/percarpercitydata/Ahmedabad - 2016 Oct/GJ20N0977_City\\Z20160616162057a.csv',
  'ha': 0,
  'hb': 0,
  'total_idle_time': 10.466666666666667,
  'overspeed': 0.0,
  'fuel_used': 0.4577,
  'Total_distance': 7.0,
  'Total_time': 31.401666666666667,
  